# Packages

In [ ]:
import cv2

from google.colab import files
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from IPython.display import clear_output, HTML
clear_output()

# Prep

In [ ]:
path = '/usr/local/share/jupyter/nbextensions/google.colab'
!cp -r {path}/* .
!rm -r {path}
!ln -s /content {path}
# change base tag

def change_base_url():
  eval_js("""
  var base = document.createElement('base')
  base.href = 'https://localhost:8080/nbextensions/google.colab/'
  document.head.prepend(base)
  """)
# make it run automatically in every cell
get_ipython().events.register('pre_run_cell', change_base_url)

fpath_img1 = 'pose_sample2.jpeg'
!gdown 1akyrQ4k_2weAN1rFOZ9xJ-ReUhqM3anb -O "$fpath_img1"

fpath_img2 = 'od_sample2.jpg'
!gdown 19UxDH_g3EHwEnz1WvLSlXe6D8ypgIwNC -O "$fpath_img2"

clear_output()

# Single Person Estimation

In [ ]:
#@title Template Image
html_code = HTML('''
<!-- 相關依賴套件載入 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-core"></script>

<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-converter"></script>

<!-- 本章節採用tfjs，因此載入TF.js backend -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-backend-webgl"></script>

<!-- 載入手勢辨識模型 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow-models/pose-detection"></script>

<!-- 用於呈現分析結果canvas -->
<canvas id='canvas1'></canvas>

<script>
// 宣告canvas 物件
const canvas1 = document.getElementById('canvas1');
// 宣告canvas 繪圖物件
var ctx = canvas1.getContext('2d')

// 先告js 影像物件
var myImage = new Image();
// 指定本地端範例影像路徑
myImage.src = "pose_sample2.jpeg";
// 一旦成功載入影像，觸發執行loadImage函式
myImage.addEventListener("load", loadImage, false);

// 觸發函式
async function loadImage(e) {
  await tf.setBackend('webgpu')
  // 範例影像大小
  img_width = myImage.width;
  img_height = myImage.height;

  // 調整canvas大小
  canvas1.width = img_width;
  canvas1.height = img_height;

  // 人體關節點註解文字顏色
  ctx.strokeStyle = "red";
  // 人體關節點註解文字大小&字體
  ctx.font = "20px Arial";
  // canvas 繪製底圖
  ctx.drawImage(myImage, 0, 0, img_width, img_height);

  // 讀取模型
  const detectorConfig = {modelType: poseDetection.movenet.modelType.SINGLEPOSE_LIGHTNING};
  const detector = await poseDetection.createDetector(poseDetection.SupportedModels.MoveNet, detectorConfig);

  // 模型推論分析
  const poses = await detector.estimatePoses(canvas1);

  // 繪製模型分析結果
  keypoints = poses[0].keypoints
  for (var i = 0; i < keypoints.length; i++) {
      console.log(keypoints[i]);

      // 關節點pixel-wise位置
      x = keypoints[i].x
      y = keypoints[i].y

      // 關節點類別
      name = keypoints[i].name

      // 圈選關節點估測位置
      ctx.beginPath();
      ctx.arc(x, y, 20, 0, 2 * Math.PI);
      ctx.stroke();

      // 繪製關節點文字註解
      ctx.fillText(name, x+30, y);
  }
}
</script>
''')

html_code

In [ ]:
#@title Custom Image
uploaded = files.upload()
fname = list(uploaded.keys())[0]

html_code2 = HTML('''
<!-- 相關依賴套件載入 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-core"></script>
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-converter"></script>

<!-- 本章節採用tfjs，因此載入TF.js backend -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs-backend-webgl"></script>

<!-- 載入手勢辨識模型 -->
<script src="https://cdn.jsdelivr.net/npm/@tensorflow-models/pose-detection"></script>

<!-- 用於呈現分析結果canvas -->
<canvas id='canvas1'></canvas>

<script>
// 宣告canvas 物件
const canvas1 = document.getElementById('canvas1');
// 宣告canvas 繪圖物件
var ctx = canvas1.getContext('2d')

// 先告js 影像物件
var myImage = new Image();
// 指定本地端範例影像路徑
myImage.src = "%s";
// 一旦成功載入影像，觸發執行loadImage函式
myImage.addEventListener("load", loadImage, false);

// 觸發函式
async function loadImage(e) {
  await tf.setBackend('webgpu')
  // 範例影像大小
  img_width = myImage.width;
  img_height = myImage.height;

  // 調整canvas大小
  canvas1.width = img_width;
  canvas1.height = img_height;

  // 人體關節點註解文字顏色
  ctx.strokeStyle = "red";
  // 人體關節點註解文字大小&字體
  ctx.font = "20px Arial";
  // canvas 繪製底圖
  ctx.drawImage(myImage, 0, 0, img_width, img_height);

  // 讀取模型
  const detectorConfig = {modelType: poseDetection.movenet.modelType.SINGLEPOSE_LIGHTNING};
  const detector = await poseDetection.createDetector(poseDetection.SupportedModels.MoveNet, detectorConfig);

  // 模型推論分析
  const poses = await detector.estimatePoses(canvas1);

  // 繪製模型分析結果
  keypoints = poses[0].keypoints
  for (var i = 0; i < keypoints.length; i++) {
      console.log(keypoints[i]);

      // 關節點pixel-wise位置
      x = keypoints[i].x
      y = keypoints[i].y

      // 關節點類別
      name = keypoints[i].name

      // 圈選關節點估測位置
      ctx.beginPath();
      ctx.arc(x, y, 20, 0, 2 * Math.PI);
      ctx.stroke();

      // 繪製關節點文字註解
      ctx.fillText(name, x+30, y);
  }
}
</script>
''' % (fname))

html_code2